In [1]:
println("================================================================================================")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.132:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1699630617671)
SparkSession available as 'spark'


In [35]:
import scala.collection.mutable
import scala.util.Try

import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

import scala.collection.mutable
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [36]:
// TODO FIX RUN NB INTO ANOTHER
//%run ./OSUtils.ipynb

def isWindows: Boolean = System.getProperty("os.name").toUpperCase.contains("WINDOWS")
def isLinux: Boolean = System.getProperty("os.name").toUpperCase.contains("LINUX")

def printOS: Unit =    
    if (isWindows) println("THIS IS A WINDOWS OS")
    else println("THIS IS A LINUX OS")

isWindows: Boolean
isLinux: Boolean
printOS: Unit


In [37]:
val dfFlights = spark.read.option("header", true).csv("./data/flights.csv")

dfFlights: org.apache.spark.sql.DataFrame = [DayofMonth: string, DayOfWeek: string ... 5 more fields]


In [38]:
dfFlights.printSchema

root
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrDelay: string (nullable = true)



In [50]:
/** Check if schema from a df contains some values.
  * @param schema: Df schema
  * @param fieldsToCheck: List of columns to check
  * @return: Unit
  */
def schemaContainsField(schema: List[String], fieldsToCheck: List[String]): Unit = {
    fieldsToCheck.foreach(f => {
        if (schema.contains(f)) println(s"$f --> true")
        else println(s"$f --> false")})
}

schemaContainsField: (schema: List[String], fieldsToCheck: List[String])Unit


In [51]:
val fieldsToCheck: List[String] = List("DayofMonth", "Carrier", "carrier") // Case sensitive!!

fieldsToCheck: List[String] = List(DayofMonth, Carrier, carrier)


In [52]:
schemaContainsField(dfFlights.schema.names.toList, fieldsToCheck)

DayofMonth --> true
Carrier --> true
carrier --> false


In [59]:
/** Check if df has single column
  * @param df: DataFrame
  * @param column:Column to check
  * @return: Boolean if df has column
*/
def hasColumn(df: DataFrame, column: String): Boolean = Try(df(column)).isSuccess

hasColumn: (df: org.apache.spark.sql.DataFrame, column: String)Boolean


In [62]:
hasColumn(dfFlights, "DayOfWeek")

res28: Boolean = true


In [60]:
hasColumn(dfFlights, "Aeroport")

res26: Boolean = false
